In [12]:
import numpy as np
import cv2
import heapq
import statistics
import math
from scipy.spatial import KDTree

In [13]:
def popularity(image,k):
        (m,n,_) = image.shape
        d = {}
        for i in range(m):
            for j in range(n):
                t = tuple(image[i,j])
                if t in d:
                    d[t] += 1
                else:
                    d[t] = 1
        top_k_colors =heapq.nlargest(k, d, key=d.get)
        for color in top_k_colors:
            print(d[color])
        return top_k_colors

In [14]:
def median(list_of_tuples,total):
    list_of_tuples = list(list_of_tuples)
    list_of_tuples.sort()
    s = 0
    i = 0
    while(s < total/2):
        s += list_of_tuples[i][1]
        i += 1
    if i==len(list_of_tuples):
        i -= 1
    return list_of_tuples[i][0]

In [15]:
def find_color(list_of_tuples):
    min_rgb = np.array([255,255,255])
    max_rgb = np.array([0,0,0])
    for i in list_of_tuples:
        min_rgb = np.minimum(min_rgb,i[0])
        max_rgb = np.maximum(max_rgb,i[0])
    diff = max_rgb-min_rgb
    return diff.argmax()

In [16]:
def split(list_of_tuples,k):
    # list_of_tuples ((r,g,b), frequencies)
    if k == 1:
        return [np.average([x[0] for x in list_of_tuples],weights=[x[1] for x in list_of_tuples],axis=0).astype(np.int32)]
    color_to_split = find_color(list_of_tuples)
    total = 0
    d = {}
    for t in list_of_tuples:
        color_val = t[0][color_to_split]
        if color_val in d:
            d[color_val] += t[1]
            total += t[1]
        else:
            d[color_val] = t[1]
            total += t[1]
    m = median(d.items(),total)
    left = []
    right = []
    for i in list_of_tuples:
        if i[0][color_to_split] < m:
            left.append(i)
        else:
            right.append(i)
    l_colors = split(left,k//2)
    r_colors = split(right,k//2)
    return l_colors+r_colors

In [17]:
def median_cut(image,k):
    (m,n,c) = image.shape
    d = {}
    for i in range(m):
        for j in range(n):
            t = tuple(image[i,j])
            if t in d:
                d[t] += 1
            else:
                d[t] = 1
    l = d.items()
    return split(l,k)

In [18]:
def popularity_quant(image, k):
    finalImage = image.copy()
    color_map = popularity(image, k)
    kd_tree = KDTree(color_map)
    data = kd_tree.data
    (m,n,_) = image.shape
    for i in range(m):
        for j in range(n):
            finalImage[i,j] = data[kd_tree.query(image[i,j])[1]]
    return finalImage

In [19]:
def get_norm(t1 , t2):
    (xa, ya, za) = t1
    (xb, yb, zb) = t2
    return (xa-xb)^2 + (ya-yb)^2 + (za-zb)^2


In [20]:
# test_image = cv2.imread('test1.png', cv2.IMREAD_GRAYSCALE)
test_image = cv2.imread('test1.png')

In [21]:
test_image.shape

(540, 470, 3)

In [22]:
def dither(image, color_map):
    # Assuming we have the color map already
    finalImage = image.copy().astype(np.float32)
    (m,n,_) = image.shape
    kd_tree = KDTree(color_map)
    data = kd_tree.data
    for i in range(m):
        for j in range(n):
            t = finalImage[i,j]
            for i in range(len(t)):
                if t[i] < 0:
                    t[i] = 0
                elif t[i] > 255:
                    t[i] = 255
            min_col = data[kd_tree.query(t)[1]]
            finalImage[i,j] = min_col
            error = t - min_col
            if(j+1 < n):
                finalImage[i,j+1] = finalImage[i, j+1] + error*(7.0/16.0)
            if(i+1 < m):
                finalImage[i+1,j] = finalImage[i+1, j] + error*(5.0/16.0)
            if(j+1 < n and i+1 < m):
                finalImage[i+1,j+1] = finalImage[i+1, j+1] + error*(1.0/16.0)
            if (i+1 < m and j-1 > -1):
                finalImage[i+1,j-1] = finalImage[i+1,j-1] + error*(3.0/16.0)
    return finalImage

In [25]:
%%time
img = dither(test_image, popularity(test_image, 8))

20792
6968
1726
1651
1492
1484
1366
1328
CPU times: user 21.7 s, sys: 231 ms, total: 22 s
Wall time: 21.9 s


In [26]:
cv2.imwrite('dither_test1.png', img)

True